This notebook will be for data cleaning

In [1]:
import pandas as pd

In [3]:
mta = pd.read_csv('https://data.ny.gov/api/views/ypri-5iwa/rows.csv?accessType=DOWNLOAD&sorting=true')

In [21]:
mta.tail()

,DATE,TIME,ENTRIES,EXITS
16233622,01/01/2021 12:00:00 AM,04:00:00,5554,541
16233623,01/01/2021 12:00:00 AM,08:00:00,5554,541
16233624,01/01/2021 12:00:00 AM,12:00:00,5554,541
16233625,01/01/2021 12:00:00 AM,16:00:00,5554,541
16233626,01/01/2021 12:00:00 AM,20:00:00,5554,541


In [5]:
#removing unnecessary columns
mta.drop(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME','DIVISION','DESC'],
        inplace=True,
        axis=1)

In [11]:
# checking for null data (horay there isn't any)
mta.isna().sum()

DATE       0
TIME       0
ENTRIES    0
EXITS      0
dtype: int64

In [19]:
#simplifying the date column (removing timestamp)
